In [96]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from time import time

# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  999


In [97]:
import os
import imageio
from matplotlib import pyplot as plt
fs = os.listdir('bam_dataset_download/downloaded/')
print(len(fs))
for f in list(fs)[:10]:
    img = imageio.imread('bam_dataset_download/downloaded/' + f)
    plt.imshow(img)
    plt.show()

0


In [98]:
original_ds = dset.ImageFolder(root='./data',
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
# Create the dataloader
original_dl = torch.utils.data.DataLoader(original_ds, batch_size=1,
                                         shuffle=False, num_workers=4)



image_size = 256
dataset = dset.ImageFolder(root='./data',
                           transform=transforms.Compose([
                               #transforms.RandomApply([
                               #    transforms.RandomAffine(0, translate=None, scale=(0.8, 1.2), shear=None)], p=0.1),
                               transforms.Resize(image_size),
                               transforms.RandomCrop(image_size),
                               #transforms.RandomApply([
                               #    transforms.RandomAffine(4, translate=None, scale=None, shear=5)], p=0.2),
                               transforms.ColorJitter(brightness=0.15, contrast=0.15, saturation=0.15, hue=0.1),
                               transforms.RandomHorizontalFlip(p=0.5),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                           ]))
# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1,
                                         shuffle=False, num_workers=4)

In [ ]:
# (bad) benchmark (keep in mind this does not include performance gains due to large batch sizes, using gpu,..)
t0 = time()

for i, data in enumerate(dataloader):
    if i > 300:
        break
    
print(time()-t0)

In [ ]:

for i, a in enumerate(zip(dataloader, original_dl)):
    if i % 25 == 7:
        transformed_img, original_img = a
        plt.imshow(((transformed_img[0][0].numpy()+1)/2).transpose((1, 2, 0)))
        plt.show()
        plt.imshow(((original_img[0][0].numpy()+1)/2).transpose((1, 2, 0)))
        plt.show()
        
    if i > 25*18:
        break

In [ ]:

image_size = 256
dataset = dset.ImageFolder(root='./data',
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.RandomCrop(image_size),
                               transforms.RandomHorizontalFlip(p=0.5),
                               transforms.ToTensor(),
                           ]))
dl2 = torch.utils.data.DataLoader(dataset, batch_size=1,
                                         shuffle=True, num_workers=1)

sum_img = np.zeros((image_size, image_size, 3))
k = 0


for i, data in enumerate(dl2):
    sum_img += data[0][0].numpy().transpose((1, 2, 0))
    
    #print(np.min(data[0][0].numpy().transpose((1, 2, 0))), np.max(data[0][0].numpy().transpose((1, 2, 0))))
    k += 1
    
    if i > 200:
        break
        
        
plt.imshow(sum_img/k)
print('min/max of overall mean image:', np.min(sum_img/k, axis=(0, 1)), np.max(sum_img/k, axis=(0, 1)))
print('mean channel values:', np.mean(sum_img/k, axis=(0, 1)))
print('std channel values (this seems to be incorrect?!):', np.std(sum_img/k, axis=(0, 1)))

In [ ]:
import os
import imageio
import random
from matplotlib import pyplot as plt
fs = list(os.listdir('cropped_TRAIN/'))
random.shuffle(fs)
print(len(fs))
for f in fs[:100]:
    img = imageio.imread('cropped_TRAIN/' + f)
    plt.imshow(img)
    plt.show()